In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import tensorflow.keras
import os
checkpoint_path = "training_ResNet_52/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
from keras.callbacks import History

In [2]:
d_rate=0.1 #The rate of droupout for the layers in the ResNet
act='gelu'#The activation function use ('relu','swish','elu','gelu','selu')
lr=0.001 #The default learning rate Default 0.01
mom=0.9 #The momentum parameter Default 0.9
test_split=1 #Default 1 and range[1,0)
#Reducing test_split will make the model faster but will also reduce the accuracy of the Validation metric
#On changing test_split run the lower initialisation cell, marked below**

In [3]:
df_data=pd.read_csv("DataSetNormalised.csv")

df_data

In [4]:
df_label=pd.read_csv("LabelSet.csv")

In [5]:
x=np.array(df_data.iloc[:,1:])

In [6]:
y=np.array(df_label.iloc[:,1])

In [7]:
x=x.astype(np.float32)

In [8]:
#On changing test_split run this--------**
size=10000000 + test_split*1000000
x_train=x[:][:10000000]
y_train=y[:][:10000000]
x_test=x[:][10000000:size]
y_test=y[:][10000000:size]

y_train.shape

In [9]:
def identity_block(X,no_nodes,Activation):#A custom Residual identity block with a skip connection
    X_shortcut=X
    #There will be 3 layers in total for the sequential connection
    #Activation will be specified by the user as per the Keras Documentation
    #no_nodes denote the size of the model which will be built
    #Applying Batch Normalisation to size the inputs
    #Layer 1 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 2-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 3-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 4-------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X])
    X=tf.keras.layers.Activation('relu')(X)
    #--------------------------------------------------------
    X_shortcut2=X
     #Layer 5 -----------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 6-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.BatchNormalization(axis=1)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    #Layer 7-------------------------------------------------
    X=tf.keras.layers.Dense(no_nodes,activation=Activation)(X)
    X=tf.keras.layers.Dropout(d_rate)(X)
    #--------------------------------------------------------
    X= tf.keras.layers.Add()([X_shortcut, X,X_shortcut2])
    #Layer 8-------------------------------------------------
    X=tf.keras.layers.Activation('relu')(X)
    return X
    

In [10]:
inputs=tf.keras.layers.Input(shape=(28,))                    #Keras input layer
X=tf.keras.layers.Dense(300,activation=act)(inputs)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)
X=identity_block(X=X,no_nodes=300,Activation=act)        
outputs=tf.keras.layers.Dense(1,activation='sigmoid')(X)     #keras output layer
model=tf.keras.Model(inputs,outputs)                         #Model declaration

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 300)          8700        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 300)          90300       ['dense[0][0]']                  
                                                                                                  
 batch_normalization (BatchNorm  (None, 300)         1200        ['dense_1[0][0]']                
 alization)                                                                                   

In [12]:
model.compile(
        optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.01,
        beta_1=mom,
        beta_2=0.999),
        loss='binary_crossentropy',
        metrics=['accuracy','AUC']
    )

In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

In [14]:
class HostLog(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        with open("log3.txt", "a") as f:
            f.write(f"at epoch {epoch}:{str(logs)}\n\n")

In [ ]:
while True:
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=True,
                                                     save_best_only=True)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_accuracy",
                                                     factor = 0.7,
                                                     patience = 5,
                                                     min_lr = 10**-6,
                                                     cooldown = 10)

    history = History()

    model.fit(
        x_train,
        y_train,
        epochs=20,
        batch_size=10240,
        validation_data=(x_test,y_test),
        callbacks=[cp_callback, reduce_lr, history, HostLog()]
    )

    model.save('ResNet_52')

    with open("sunnyMamoni_m2.txt", "a") as f:
        f.write(str(history.history)+"\n\n\n\n")

Epoch 1/20
977/977 [==============================] - ETA: 0s - loss: 0.4452 - accuracy: 0.7853 - auc: 0.8709
Epoch 00001: val_loss improved from inf to 2.16567, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 252s 249ms/step - loss: 0.4452 - accuracy: 0.7853 - auc: 0.8709 - val_loss: 2.1657 - val_accuracy: 0.7846 - val_auc: 0.8701 - lr: 6.0025e-04
Epoch 2/20
977/977 [==============================] - ETA: 0s - loss: 0.4451 - accuracy: 0.7852 - auc: 0.8710
Epoch 00002: val_loss improved from 2.16567 to 0.89279, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 242s 248ms/step - loss: 0.4451 - accuracy: 0.7852 - auc: 0.8710 - val_loss: 0.8928 - val_accuracy: 0.7845 - val_auc: 0.8700 - lr: 6.0025e-04
Epoch 3/20
977/977 [==============================] - ETA: 0s - loss: 0.4449 - accuracy: 0.7854 - auc: 0.8711
Epoch 00003: val_loss improved from 0.89279 to 0.52051, saving model to training_ResNet_52\cp.ckpt
977/977 [

C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 1/20
977/977 [==============================] - ETA: 0s - loss: 0.4432 - accuracy: 0.7865 - auc: 0.8722
Epoch 00001: val_loss improved from inf to 0.55537, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 243s 248ms/step - loss: 0.4432 - accuracy: 0.7865 - auc: 0.8722 - val_loss: 0.5554 - val_accuracy: 0.7847 - val_auc: 0.8702 - lr: 4.2018e-04
Epoch 2/20
977/977 [==============================] - ETA: 0s - loss: 0.4430 - accuracy: 0.7866 - auc: 0.8723
Epoch 00002: val_loss improved from 0.55537 to 0.44625, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 242s 248ms/step - loss: 0.4430 - accuracy: 0.7866 - auc: 0.8723 - val_loss: 0.4463 - val_accuracy: 0.7849 - val_auc: 0.8703 - lr: 4.2018e-04
Epoch 3/20
977/977 [==============================] - ETA: 0s - loss: 0.4429 - accuracy: 0.7867 - auc: 0.8724
Epoch 00003: val_loss did not improve from 0.44625
977/977 [==============================] - 242s 247ms/ste

C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 1/20
977/977 [==============================] - ETA: 0s - loss: 0.4418 - accuracy: 0.7874 - auc: 0.8731
Epoch 00001: val_loss improved from inf to 0.52425, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 243s 248ms/step - loss: 0.4418 - accuracy: 0.7874 - auc: 0.8731 - val_loss: 0.5243 - val_accuracy: 0.7852 - val_auc: 0.8707 - lr: 2.9412e-04
Epoch 2/20
977/977 [==============================] - ETA: 0s - loss: 0.4418 - accuracy: 0.7873 - auc: 0.8731
Epoch 00002: val_loss improved from 0.52425 to 0.47132, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 242s 248ms/step - loss: 0.4418 - accuracy: 0.7873 - auc: 0.8731 - val_loss: 0.4713 - val_accuracy: 0.7848 - val_auc: 0.8704 - lr: 2.9412e-04
Epoch 3/20
977/977 [==============================] - ETA: 0s - loss: 0.4417 - accuracy: 0.7873 - auc: 0.8731
Epoch 00003: val_loss did not improve from 0.47132
977/977 [==============================] - 242s 247ms/ste

C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 1/20
977/977 [==============================] - ETA: 0s - loss: 0.4409 - accuracy: 0.7880 - auc: 0.8736
Epoch 00001: val_loss improved from inf to 0.44567, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 243s 248ms/step - loss: 0.4409 - accuracy: 0.7880 - auc: 0.8736 - val_loss: 0.4457 - val_accuracy: 0.7853 - val_auc: 0.8707 - lr: 2.0589e-04
Epoch 2/20
977/977 [==============================] - ETA: 0s - loss: 0.4407 - accuracy: 0.7880 - auc: 0.8738
Epoch 00002: val_loss did not improve from 0.44567
977/977 [==============================] - 242s 247ms/step - loss: 0.4407 - accuracy: 0.7880 - auc: 0.8738 - val_loss: 0.4461 - val_accuracy: 0.7850 - val_auc: 0.8707 - lr: 2.0589e-04
Epoch 3/20
977/977 [==============================] - ETA: 0s - loss: 0.4408 - accuracy: 0.7879 - auc: 0.8737
Epoch 00003: val_loss did not improve from 0.44567
977/977 [==============================] - 242s 247ms/step - loss: 0.4408 - accuracy: 0.7879 - auc: 0.873

C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
C:\Users\ML_RTC\anaconda3\lib\site-packages\keras\saving\saved_model\layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


Epoch 1/20
977/977 [==============================] - ETA: 0s - loss: 0.4399 - accuracy: 0.7885 - auc: 0.8743
Epoch 00001: val_loss improved from inf to 0.47775, saving model to training_ResNet_52\cp.ckpt
977/977 [==============================] - 243s 248ms/step - loss: 0.4399 - accuracy: 0.7885 - auc: 0.8743 - val_loss: 0.4777 - val_accuracy: 0.7855 - val_auc: 0.8709 - lr: 1.0088e-04
Epoch 2/20
977/977 [==============================] - ETA: 0s - loss: 0.4401 - accuracy: 0.7884 - auc: 0.8742
Epoch 00002: val_loss did not improve from 0.47775
977/977 [==============================] - 241s 247ms/step - loss: 0.4401 - accuracy: 0.7884 - auc: 0.8742 - val_loss: 1.1941 - val_accuracy: 0.7855 - val_auc: 0.8710 - lr: 1.0088e-04
Epoch 3/20
977/977 [==============================] - ETA: 0s - loss: 0.4400 - accuracy: 0.7884 - auc: 0.8742
Epoch 00003: val_loss did not improve from 0.47775
977/977 [==============================] - 242s 248ms/step - loss: 0.4400 - accuracy: 0.7884 - auc: 0.874

model.save('ResNet_52')

with open("sunnyMamoni_m2.txt", "a") as f:
    f.write(str(history.history))